In [4]:
from mpl_rcParams import * #Upload the usual settings that we use for publication grade figures!

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import unitary_group

from scaling_lib.misc_utils import custom_mkdir, timestring
from scaling_lib.mode_solver import solve_modes
from scaling_lib.simulation_tdwg_repo import WaveguideSimulationWithBackground
from scaling_lib.deltaN import U2A
from scaling_lib.misc_helpers import add_absorbing_boundary, smoothen_waveguide
from scaling_lib.sweeps import sweep_product
from scaling_lib.ftutils_torch import fft_centered_ortho, ft_f_axis, ifft_centered_ortho
import scaling_lib.pnn_utils as pnn_utils 
import scaling_lib.ftutils_np as ftutils

import torch
import torch.fft as fft
from fft_conv_pytorch import fft_conv, FFTConv2d
import copy

import torch.nn as nn
import torch.optim as optim

In [5]:
class TDwgNet(nn.Module):
    def __init__(self, wg, delta_n_val, background_dn, Evecs, betas, dx, d_wg, k_cprog, device):
        super(TDwgNet, self).__init__()

        self.d_wg = d_wg
        Nx_wg = wg.x2ind(d_wg/2) - wg.x2ind(-d_wg/2)
        mod = torch.zeros([wg.Nz, Nx_wg], requires_grad=True, dtype=torch.float32).to(device)
        self.mod = pnn_utils.Parameter(mod, limits=[-0.45, 0.45]) 
        self.background_dn = background_dn.to(device)
        self.delta_n_val = delta_n_val
        self.input_beams = torch.from_numpy(Evecs).to(dtype=torch.complex128).to(device)
        self.output_modes = torch.from_numpy(Evecs).to(dtype=torch.complex128).to(device)*dx
        self.wg = wg
        self.betas = torch.tensor(betas, dtype=torch.float32).to(device)
        self.k_cprog = k_cprog
        self.device = device

    def forward(self, indices = None, fast_flag=True, ind=0):
        self.wg.set_background_delta_n(self.background_dn)
        
        delta_n = torch.zeros([self.wg.Nz, self.wg.Nx], device = self.device)
        mod = self.mod
        mod = mod.clip(-0.5, 0.5)
        delta_n[:, self.wg.x2ind(-self.d_wg/2):self.wg.x2ind(self.d_wg/2)] = mod
        delta_n = self.delta_n_val*self.smoothen(delta_n)
        
        if fast_flag:
            output_beams = self.wg.run_simulation(self.input_beams[indices], delta_n)
        else:
            output_beams = self.wg.run_simulation_slow(self.input_beams[ind, :])
        
        a_out = output_beams@self.output_modes.T
        U = (a_out.T*torch.exp(-1j*self.betas[indices]*self.wg.Lz)).T
        return U
        
    def smoothen(self, mod):
        # self.mod: (Nz, Nx)
        Nz, Nx = mod.shape
        dz = self.wg.z_axis[1] - self.wg.z_axis[0]
        dx = self.wg.x_axis[1] - self.wg.x_axis[0]
    
        # Frequency axes
        kz = torch.fft.fftfreq(Nz, d=dz).to(self.device) * 2 * torch.pi  # angular frequency
        kx = torch.fft.fftfreq(Nx, d=dx).to(self.device) * 2 * torch.pi
    
        # Create 2D meshgrid for frequencies
        Kz, Kx = torch.meshgrid(kz, kx, indexing='ij')
    
        # Construct Gaussian filter: exp(- (k^2 / self.k_cprog^2))
        gaussian_filter = torch.exp(-((Kz**2 + Kx**2) / (self.k_cprog**2)))
    
        # FFT of the mod parameter
        mod_fft = torch.fft.fft2(mod)
    
        # Apply the filter
        mod_fft_filtered = mod_fft * gaussian_filter
    
        # Inverse FFT to get back to spatial domain
        mod_filtered = torch.fft.ifft2(mod_fft_filtered).real
    
        return mod_filtered
    
def L2(p, q):
    x = torch.abs(p-q)**2
    return x.sum(dim = -1).mean()/2

def run_training_loop(tdwg_pnn, U_target, iterations, lr, batch_size, device, gamma=0.99, print_interval=1):
    optimizer = optim.Adam(tdwg_pnn.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=gamma)
    U_target = U_target.to(device)
    loss_list = []
    
    n = len(tdwg_pnn.input_beams)  # length of the original vector
    indices = np.arange(n)
    batch_indices = np.array_split(indices, np.ceil(n / batch_size))  # Split into subvectors

    tdwg_pnn.train()
    for i in range(iterations):
        optimizer.zero_grad()

        for indices in batch_indices:
            U_num = tdwg_pnn.forward(indices)
            l_objective = L2(U_target[indices], U_num)
    
            l_lagrange = pnn_utils.lagrangian(tdwg_pnn, lag_amp = 1, factor = 20)
            l = l_objective + l_lagrange 
            # l = l_objective
            l.backward()
            
        optimizer.step()
        scheduler.step()
        loss_list.append(l.detach().cpu().data)

        if i % print_interval == 0:
            print(timestring() + f"--Iteration: {i}, Loss: {l.detach().cpu().data:.5f}")
            # get_cpu_memory_usage(threshold_gb = 0.01)
            # get_gpu_memory_usage(threshold_gb = 0.01)
        if has_converged(loss_list):
            print(timestring() + f"Converged at iteration: {i}, Loss: {l.detach().cpu().data:.5f}")
            break
    return loss_list


def has_converged(loss_list, window=5, std_threshold=0.005):
    """
    Check if training has converged based on the standard deviation of recent loss values.

    Args:
        loss_list (list): List of loss values per epoch.
        window (int): Number of recent epochs to consider.
        std_threshold (float): Maximum acceptable standard deviation.

    Returns:
        bool: True if loss has converged, False otherwise.
    """
    if len(loss_list) < window:
        return False  # Not enough data

    recent_losses = loss_list[-window:]
    std_dev = np.std(recent_losses)
    mean = np.mean(recent_losses)
    std_dev_rel = std_dev / mean

    return std_dev_rel < std_threshold  # Converged if std deviation is very small

In [6]:
def permutation_matrix(permutation):
  """
  Generates a permutation matrix from a given permutation.

  Args:
    permutation: A list or numpy array representing the permutation.

  Returns:
    A numpy array representing the permutation matrix.
  """
  n = len(permutation)
  matrix = np.zeros((n, n), dtype=int)
  for i, j in enumerate(permutation):
    matrix[i, j] = 1
  return matrix

In [7]:
def avg_error_invdes(seed, fresnel, k_cprog, Nmodes, Lz, unitary_type = 'haar', waveguide_type = 'SI', device = 'cuda:0'):
    iterations = 200
    lr = 0.5e-1
    batch_size = 10
    
    ### Set up waveguide ###
    λ0 = 1.55 #um
    ncore = 1.8
    nclad = 1.7 #a very shallow partial etch that gives 0.05 deltaN - which I think is about a 100nm partial etch, which can be done decently accurately...
    dx = 0.2
    delta_n_val = 5e-3
    
    Nz = int(np.floor(Lz/dz))
    dn_wg = ncore-nclad
    k0 = 2*np.pi/λ0
    ### Set up index distribution ###
    if waveguide_type == 'SI': d_wg = 1.5*Nmodes #3 #so that it is less than the 2 mode cut-off
    if waveguide_type == 'GRIN': d_wg = 2*Nmodes #3 #so that it is less than the 2 mode cut-off
    Lx = d_wg + 40 
    Nx = int(Lx/dx/2)*2
    z_axis = (np.arange(Nz)+1)*dz
    x_axis = ftutils.ft_t_axis(Nx, dx)
    x2ind = lambda x: np.argmin(np.abs(x_axis-x))

    ### Set up index distribution ###
    if waveguide_type == 'SI':
        dn_wg_slice = np.zeros(Nx)
        dn_wg_slice[x2ind(-d_wg/2):x2ind(d_wg/2)] = dn_wg
        dn_wg_slice = smoothen_waveguide(x_axis, dn_wg_slice, k0*nclad) 
    if waveguide_type == 'GRIN':
        dn_wg_slice = dn_wg * (1 - (2*x_axis/d_wg)**2)
        dn_wg_slice = np.clip(dn_wg_slice, 0, dn_wg)
    dn_wg_slice = add_absorbing_boundary(x_axis, dn_wg_slice, k0)
    
    betas, Evecs = solve_modes(x_axis, nclad, k0, dn_wg_slice, Nmodes, fresnel = fresnel)
    betas = betas - nclad * k0
    
    if unitary_type == 'haar': U_target = unitary_group.rvs(Nmodes, random_state=seed)
    if unitary_type == 'permutation': 
        np.random.seed(seed)
        random_permutation = np.random.permutation(Nmodes)
        U_target = permutation_matrix(random_permutation)
    A = U2A(U_target, k0, Lz)
    
    ### Inverse design ###
    U_target_torch = torch.tensor(U_target)
    
    wg = WaveguideSimulationWithBackground(nclad, x_axis, z_axis, fresnel = fresnel)
    
    tdwg_pnn = TDwgNet(wg, delta_n_val, torch.from_numpy(dn_wg_slice), Evecs, betas, dx, d_wg, k_cprog, device = device)
    
    loss_list = run_training_loop(tdwg_pnn, U_target_torch, iterations, lr, batch_size, print_interval=10, device = device)
    

    all_batches = []  # List to store batches
    n = len(tdwg_pnn.input_beams)  # length of the original vector
    indices = np.arange(n)
    batch_indices = np.array_split(indices, np.ceil(n / batch_size))  # Split into subvectors

    with torch.no_grad():
        for indices in batch_indices:
            U_num_batch = tdwg_pnn.forward(indices).cpu().numpy()  # Shape: [batch_size, n]
            all_batches.append(U_num_batch)  # Collect batch
    
    # Concatenate along the first axis (rows)
    U_num_inv = np.vstack(all_batches)  # Final shape: [n, n]
    
    avg_error_inv = np.mean((np.abs(np.abs(U_num_inv)-np.abs(U_target))))/np.mean(np.abs(U_target)) * 100

    print(f'Saved {waveguide_type}_unitary={unitary_type}_Nmodes={Nmodes}_Lz={Lz}_kcprog={k_cprog:.2e}_fresnel={fresnel}_seed={seed}.npz')
    print(f"Average error for inverse design: {avg_error_inv:.2f}%")
    np.savez(f'parameter_sweep/{waveguide_type}_unitary={unitary_type}_Nmodes={Nmodes}_Lz={Lz}_kcprog={k_cprog:.2e}_fresnel={fresnel}_seed={seed}.npz', λ0 = λ0, ncore = ncore, nclad = nclad, dn_wg = dn_wg, k0 = k0, d_wg = d_wg, Lx = Lx, Nx = Nx, dz = dz, Lz = Lz, Nz = Nz, z_axis = z_axis, dx = dx, x_axis = x_axis, dn_wg_slice = dn_wg_slice, betas = betas, Evecs = Evecs, seed = seed, U_target = U_target, U_num_inv = U_num_inv, A = A, avg_error_inv = avg_error_inv, iterations = iterations, loss_list = loss_list, lr = lr, background_dn = tdwg_pnn.background_dn.cpu(), dn_prog = tdwg_pnn.mod.detach().cpu())
    return avg_error_inv

In [8]:
dz = 0.4
Lz_list = np.logspace(np.log10(1e1), np.log10(5e3), num=12)
Lz_list = np.round(Lz_list/dz)*dz
Nmodes_list = np.logspace(np.log10(2), np.log10(200), num=10)
Nmodes_list = np.unique(np.round(Nmodes_list).astype(int))
k_cprog_list = [1.7*2*np.pi/1.55]
fresnel_list = [False]
unitary_type_list = ['haar', 'permutation']
waveguide_type_list = ['SI', 'GRIN']
seed_list = [0]

In [9]:
avg_error_invdes_mat = sweep_product(avg_error_invdes, seed_list, fresnel_list, k_cprog_list, Nmodes_list, Lz_list, unitary_type_list, waveguide_type_list)

2025-04-25--22-41-11--Iteration: 0, Loss: 0.26256


  0% 0 of 120 |                                         | Elapsed Time: 0:00:00

2025-04-25--22-41-11--Iteration: 10, Loss: 0.24512
2025-04-25--22-41-11Converged at iteration: 12, Loss: 0.24250
Saved SI_unitary=haar_Nmodes=2_Lz=10.0_kcprog=6.89e+00_fresnel=False_seed=0.npz
Average error for inverse design: 42.78%
2025-04-25--22-41-12--Iteration: 0, Loss: 0.26267
2025-04-25--22-41-12--Iteration: 10, Loss: 0.22629
2025-04-25--22-41-12Converged at iteration: 18, Loss: 0.22125


  1% 2 of 120 |                                         | Elapsed Time: 0:00:00

Saved SI_unitary=haar_Nmodes=2_Lz=17.6_kcprog=6.89e+00_fresnel=False_seed=0.npz
Average error for inverse design: 41.00%
2025-04-25--22-41-12--Iteration: 0, Loss: 0.26288
2025-04-25--22-41-13--Iteration: 10, Loss: 0.19566


...

Saved SI_unitary=haar_Nmodes=200_Lz=1615.2_kcprog=6.89e+00_fresnel=False_seed=0.npz
Average error for inverse design: 48.11%
2025-04-26--14-13-54--Iteration: 0, Loss: 0.99899
2025-04-26--14-26-09--Iteration: 10, Loss: 0.46874
2025-04-26--14-38-24--Iteration: 20, Loss: 0.30980
2025-04-26--14-51-04--Iteration: 30, Loss: 0.23734
2025-04-26--15-03-50--Iteration: 40, Loss: 0.18972
2025-04-26--15-16-36--Iteration: 50, Loss: 0.16315
2025-04-26--15-29-18--Iteration: 60, Loss: 0.14985
2025-04-26--15-42-05--Iteration: 70, Loss: 0.14222
2025-04-26--15-48-27Converged at iteration: 75, Loss: 0.13969


 99% 119 of 120 |##################################### | Elapsed Time: 17:07:40

Saved SI_unitary=haar_Nmodes=200_Lz=2842.0_kcprog=6.89e+00_fresnel=False_seed=0.npz
Average error for inverse design: 28.77%
2025-04-26--15-52-03--Iteration: 0, Loss: 0.99812
2025-04-26--16-20-35--Iteration: 10, Loss: 0.58436
2025-04-26--16-49-10--Iteration: 20, Loss: 0.37322
2025-04-26--17-17-48--Iteration: 30, Loss: 0.25069
2025-04-26--17-46-31--Iteration: 40, Loss: 0.17113
2025-04-26--18-15-08--Iteration: 50, Loss: 0.10540
2025-04-26--18-43-46--Iteration: 60, Loss: 0.06966
2025-04-26--19-12-29--Iteration: 70, Loss: 0.04806
2025-04-26--19-40-57--Iteration: 80, Loss: 0.03506
2025-04-26--20-09-34--Iteration: 90, Loss: 0.02700
2025-04-26--20-38-13--Iteration: 100, Loss: 0.02236
2025-04-26--21-06-21--Iteration: 110, Loss: 0.01962
2025-04-26--21-34-51--Iteration: 120, Loss: 0.01792
2025-04-26--22-03-29--Iteration: 130, Loss: 0.01680
2025-04-26--22-32-07--Iteration: 140, Loss: 0.01600
2025-04-26--23-00-48--Iteration: 150, Loss: 0.01541
2025-04-26--23-03-41Converged at iteration: 151, Loss:

100% 120 of 120 |################################| Elapsed Time: 1 day, 0:23:20


Saved SI_unitary=haar_Nmodes=200_Lz=5000.0_kcprog=6.89e+00_fresnel=False_seed=0.npz
Average error for inverse design: 5.78%
